In [ ]:
# Import required libraries
import requests
import pandas as pd
import json
from typing import List, Dict, Any
import time
import os

# TradePort API credentials
API_USER = "cyber"
API_KEY = "Ep6xO5U.974416bcaa0b8b0e08f15f5f03005cf7"
API_ENDPOINT = "https://api.indexer.xyz/graphql"

In [ ]:
def fetch_verified_nft_collections(offset: int = 0) -> List[Dict[str, Any]]:
    """
    Fetch verified NFT collections from TradePort API with a specific offset
    
    Args:
        offset: The offset to start fetching from
        
    Returns:
        List of collection dictionaries
    """
    headers = {
        "x-api-user": API_USER,
        "x-api-key": API_KEY,
        "Content-Type": "application/json"
    }
    
    # Query for verified collections with fixed limit of 25
    query = """
    query {
      sui {
        collections(where: {verified: {_eq: true}}, limit: 25, offset: %d) {
          title
          slug
          verified
        }
      }
    }
    """ % offset
    
    try:
        print(f"Fetching verified NFT collections with offset {offset}...")
        response = requests.post(
            API_ENDPOINT,
            headers=headers,
            json={"query": query}
        )
        
        # Print response for debugging
        print(f"Response status code: {response.status_code}")
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            return []
        
        data = response.json()
        
        # Check if we have collections in the response
        collections = data.get("data", {}).get("sui", {}).get("collections", [])
        
        print(f"Fetched {len(collections)} collections with offset {offset}")
        return collections
        
    except Exception as e:
        print(f"Error fetching collections: {e}")
        return []

In [ ]:
csv_path = "1_all_nft_types.csv"
existing_df = None

if os.path.exists(csv_path):
    print(f"Found existing CSV file: {csv_path}")
    existing_df = pd.read_csv(csv_path)
    print(f"Existing file contains {len(existing_df)} collections")
else:
    print(f"No existing CSV file found at {csv_path}")

Found existing CSV file: 1_all_nft_types.csv
Existing file contains 145 collections


In [6]:
all_collections = []

# First batch
for offset in [0, 25, 50]:
    collections = fetch_verified_nft_collections(offset)
    all_collections.extend(collections)
    # Add a small delay to avoid rate limiting
    time.sleep(1)

print(f"Total collections fetched in first batch: {len(all_collections)}")


Fetching verified NFT collections with offset 0...
Error fetching collections: HTTPSConnectionPool(host='api.indexer.xyz', port=443): Max retries exceeded with url: /graphql (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10fd40d00>: Failed to resolve 'api.indexer.xyz' ([Errno 8] nodename nor servname provided, or not known)"))
Fetching verified NFT collections with offset 25...
Error fetching collections: HTTPSConnectionPool(host='api.indexer.xyz', port=443): Max retries exceeded with url: /graphql (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10fd40250>: Failed to resolve 'api.indexer.xyz' ([Errno 8] nodename nor servname provided, or not known)"))
Fetching verified NFT collections with offset 50...
Error fetching collections: HTTPSConnectionPool(host='api.indexer.xyz', port=443): Max retries exceeded with url: /graphql (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x106e7cd00>: Failed t

In [ ]:
for offset in [75, 100, 125, 150, 175, 200]:
    collections = fetch_verified_nft_collections(offset)
    all_collections.extend(collections)
    # Add a small delay to avoid rate limiting
    time.sleep(1)

print(f"Total collections fetched after second batch: {len(all_collections)}")

Added 145 existing slugs to the seen set
New unique verified collections: 0


In [ ]:
unique_collections = []
seen_slugs = set()

# If we have existing data, add those slugs to the seen set
if existing_df is not None:
    for slug in existing_df["nft_type"]:
        seen_slugs.add(slug)
    print(f"Added {len(seen_slugs)} existing slugs to the seen set")

# Process new collections
for collection in all_collections:
    slug = collection.get("slug", "")
    if slug and slug not in seen_slugs:
        seen_slugs.add(slug)
        unique_collections.append(collection)

print(f"New unique verified collections: {len(unique_collections)}")

No new verified collections found.
Keeping existing 145 collections in the CSV file.


In [ ]:
if not unique_collections:
    print("No new verified collections found.")
    if existing_df is not None:
        print(f"Keeping existing {len(existing_df)} collections in the CSV file.")
    new_df = pd.DataFrame(columns=["nft_collection_name", "nft_type", "verified", "tradeport_url"])
else:
    # Process collections into a DataFrame
    processed_data = []
    
    for collection in unique_collections:
        # For nft_type, we'll use the slug as a fallback since we couldn't get the type directly
        slug = collection.get("slug", "")
        processed_data.append({
            "nft_collection_name": collection.get("title", ""),
            "nft_type": slug,  # Using slug as a fallback for type
            "verified": collection.get("verified", True),  # Should always be True
            "tradeport_url": f"https://www.tradeport.xyz/sui/collection/{slug}"
        })
    
    # Create DataFrame for new collections
    new_df = pd.DataFrame(processed_data)
    print(f"Created DataFrame with {len(new_df)} new collections")

# Display first few rows of new data
if not new_df.empty:
    print("\nFirst 5 rows of the new data:")
    display(new_df.head())


No collections to save.
Data saved to 1_all_nft_types.csv
Total collections in CSV: 145


In [ ]:
if existing_df is not None and not new_df.empty:
    merged_df = pd.concat([existing_df, new_df], ignore_index=True)
    print(f"\nTotal verified collections after merge: {len(merged_df)}")
    nft_df = merged_df
else:
    if not new_df.empty:
        print(f"\nTotal new verified collections: {len(new_df)}")
        nft_df = new_df
    else:
        print("No collections to save.")
        nft_df = existing_df if existing_df is not None else pd.DataFrame(
            columns=["nft_collection_name", "nft_type", "verified", "tradeport_url"]
        )

# Save to CSV if we have data
if not nft_df.empty:
    nft_df.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")
    print(f"Total collections in CSV: {len(nft_df)}")
